In [2]:
import torch
from torchvision import transforms, datasets
from torchvision.transforms import CenterCrop
from torch.utils.data import DataLoader, TensorDataset, Dataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

trainset = datasets.ImageFolder(root="../../datasets/flowers/", transform=data_transform)

In [4]:
trainset

Dataset ImageFolder
    Number of datapoints: 4323
    Root Location: ../../datasets/flowers/
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(32, 32), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                         )
    Target Transforms (if any): None

In [5]:
from utils.data import sampler

batch_size = 64

trainloader = DataLoader(
    dataset = trainset, 
    #sampler = sampler.StratifiedSampler(data_source=trainset, n_splits=5, random_state=69, sample_train=True),
    #batch_size = batch_size,
)

valloader = DataLoader(
    dataset = trainset, 
    #sampler = sampler.StratifiedSampler(data_source=trainset, n_splits=5, random_state=69, sample_train=False),
    #batch_size = batch_size,
)

classes_name = trainset.classes

SyntaxError: positional argument follows keyword argument (sampler.py, line 6)

In [6]:
import torch.nn as nn

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

LeNet = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    Flatten(),
    nn.Linear(in_features=16*5*5, out_features=120),
    nn.Linear(in_features=120, out_features=84),
    nn.Linear(in_features=84, out_features=10),
    )

LeNet.to(device)
print(LeNet)

Sequential(
  (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten()
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Linear(in_features=120, out_features=84, bias=True)
  (9): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
# define loss function and optimizer

import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(LeNet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# training

from tqdm import tqdm

loss_history = []
accuracy_history = []

for epoch in tqdm(range(10)):
    classes_correct = list(0. for i in range(5))
    classes_total = list(0. for i in range(5))
    
    for i, train_pixels in enumerate(trainloader):
        
        # take input
        pixels, label = train_pixels
        
        # make sure gradient is 0
        optimizer.zero_grad()
        
        # forward pass
        pixels = pixels.to(device)
        prediction = LeNet(pixels)
        
        # calculate loss & record it 
        loss_score = loss_fn(prediction, label)
        
        if i == (trainloader.__len__()-1): 
            loss_history.append(loss_score)
        
        # calculate accuracy & record it
        _, prediction_label = torch.max(prediction, dim=1)
        
        for i,j in zip(label,prediction_label):
            if i == j:
                classes_correct[i] += 1
            classes_total[i] += 1
        
        # backprop
        loss_score.backward()
        
        # update weight 
        optimizer.step()
    
    accuracy_per_epoch = sum(classes_correct)/sum(classes_total)
    accuracy_history.append(accuracy_per_epoch)
        

In [ ]:
for i,(j,k) in enumerate(zip(classes_correct, classes_total)):
    print("Accuracy of %s is %d%s" % (classes_name[i], j/k*100, "%"))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(loss_history, label="cross entropy loss");
plt.plot(accuracy_history, label="accuracy")
plt.xlabel('iterations');
plt.ylabel('metric');
plt.legend();

In [ ]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))

# validation
with torch.no_grad():
    for i, val_pixels in tqdm(enumerate(valloader)):
        
        # take pixels
        pixels, label = val_pixels
        
        # forward pass
        prediction = LeNet(pixels)
        
        # count accuracy
        _,prediction_label = torch.max(prediction, dim=1)
        for i,j in zip(label, prediction_label):
            if i==j:
                class_correct[i] += 1
            class_total[i] += 1


In [ ]:
for i,(j,k) in enumerate(zip(class_correct, class_total)):
    print("Accuracy of %s is: %d%s"  % (classes_name[i], j/k*100, "%"))

In [ ]:
trainloader.__len__()